In [1]:
import os
import numpy as np
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
import tensorflow as tf
from keras import backend as K
from keras.losses import binary_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from skimage.morphology import label
import matplotlib.pyplot as plt
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
import random
from u_net import Unet, get_unet_256
from skimage import io

Using TensorFlow backend.


In [2]:
def make_df(train_path, test_path, mask_path, img_size):
    train_ids = next(os.walk(train_path))[2]
    test_ids = next(os.walk(test_path))[2]
    X_train = np.zeros((len(train_ids), img_size, img_size, 3), dtype=np.uint8)
    Y_train = np.zeros((len(train_ids), img_size, img_size, 1), dtype=np.bool)
    for i, id_ in enumerate(train_ids):
        path = train_path + '/' + id_
        img = cv2.imread(path)
        img = cv2.resize(img, (img_size, img_size))
        X_train[i] = img

        m_path = mask_path + '/' + id_
        msk = io.imread(m_path, plugin='pil')
        # msk = cv2.imread(m_path, 0)
        msk = cv2.resize(msk, (img_size, img_size))
        msk = msk[..., np.newaxis]
        #msk = np.expand_dims(msk,axis = -1)
        Y_train[i] = msk

    X_test = np.zeros((len(test_ids), img_size, img_size, 3), dtype=np.uint8)
    # sizes_test = []
    for i, id_ in enumerate(test_ids):
        path = test_path + '/' + id_
        imgTest = cv2.imread(path)
        imgTest = cv2.resize(imgTest, (img_size, img_size))
        X_test[i] = imgTest

    return X_train, Y_train, X_test

In [3]:
def generator(xtr, xval, ytr, yval, batch_size):
    data_gen_args = dict(horizontal_flip=True,
                         vertical_flip=True,
                         rotation_range=90.,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         zoom_range=0.2,
                         fill_mode = "constant",
                         cval = 0.)

    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)
    image_datagen.fit(xtr, seed=7)
    mask_datagen.fit(ytr, seed=7)
    image_generator = image_datagen.flow(xtr, batch_size=batch_size, seed=7)
    mask_generator = mask_datagen.flow(ytr, batch_size=batch_size, seed=7)
    train_generator = zip(image_generator, mask_generator)

    val_gen_args = dict()
    image_datagen_val = ImageDataGenerator(**val_gen_args)
    mask_datagen_val = ImageDataGenerator(**val_gen_args)
    image_datagen_val.fit(xval, seed=7)
    mask_datagen_val.fit(yval, seed=7)
    image_generator_val = image_datagen.flow(xval, batch_size=batch_size, seed=7)
    mask_generator_val = mask_datagen.flow(yval, batch_size=batch_size, seed=7)
    val_generator = zip(image_generator_val, mask_generator_val)

    return train_generator, val_generator


In [4]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec))

In [5]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

In [6]:
def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

In [7]:
def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

In [8]:
def pred():
    preds_test = model.predict(X_test, verbose=1)
    preds_test_t = (preds_test > 0.5).astype(np.uint8)

    preds_test_upsampled = []
    for i in range(len(preds_test)):
        preds_test_upsampled.append(cv2.resize(preds_test[i],
                                               (sizes_test[i][0], sizes_test[i][1])).transpose(1, 0))

    test_ids = next(os.walk(test_path))[1]
    new_test_ids = []
    rles = []
    for n, id_ in enumerate(test_ids):
        rle = list(prob_to_rles(preds_test_upsampled[n]))
        rles.extend(rle)
        new_test_ids.extend([id_] * len(rle))
    sub = pd.DataFrame()
    sub['ImageId'] = new_test_ids
    sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
    sub.to_csv('sub_aug_512.csv', index=False)


if __name__ == "__main__":
    img_size = 256
    batch_size =4
    train_path = 'Input/train/'
    test_path = 'Input/test/'
    mask_path = 'Input/mask/'

    X_train, Y_train, X_test = make_df(train_path, test_path, mask_path, img_size)
    xtr, xval, ytr, yval = train_test_split(X_train, Y_train, test_size=0.1, random_state=7)
    train_generator, val_generator = generator(xtr, xval, ytr, yval, batch_size)

    model = get_unet_256(input_shape=(img_size, img_size, 3),num_classes=1)
    model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[mean_iou, dice_coeff])

    callbacks = [EarlyStopping(monitor='val_mean_iou',
                               patience=50,
                               verbose=2,
                               min_delta=1e-4,
                               mode='max'
                                ),
                 ReduceLROnPlateau(monitor='val_mean_iou',
                                   factor=np.sqrt(0.1),
                                   patience=5,
                                   cooldown=2,
                                   verbose=1,
                                   epsilon=1e-4,
                                   mode='max'
                                    ),
                 ModelCheckpoint(monitor='val_mean_iou',
                                 filepath='weights/unet_256_arghya.{epoch:02d}-{val_loss:.5f}-{val_mean_iou:.5f}.h5',
                                 save_best_only=True,
                                 save_weights_only=False,
                                 mode='max',
                                 #period = 1
                                  ),
                 TensorBoard(log_dir="logs/",  histogram_freq=0,  write_graph=True, write_images=True)]
    model.fit_generator(train_generator,
                        steps_per_epoch=len(xtr) / 6,
                        epochs=300,
                        validation_data=val_generator,
                        validation_steps=len(xval) / batch_size,
                        callbacks=callbacks)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


F:\Running_file\Anaconda\lib\site-packages\keras\callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 1/300
45/45 [==============================] - ETA: 23:07 - loss: 1.2607 - mean_iou: 0.0000e+00 - dice_coeff: 0.55 - ETA: 20:52 - loss: 1.1102 - mean_iou: 0.0907 - dice_coeff: 0.5997   - ETA: 19:49 - loss: 1.0485 - mean_iou: 0.1338 - dice_coeff: 0.62 - ETA: 19:00 - loss: 1.0176 - mean_iou: 0.1631 - dice_coeff: 0.64 - ETA: 18:26 - loss: 1.0003 - mean_iou: 0.1855 - dice_coeff: 0.65 - ETA: 17:52 - loss: 0.9939 - mean_iou: 0.2030 - dice_coeff: 0.66 - ETA: 17:24 - loss: 0.9653 - mean_iou: 0.2160 - dice_coeff: 0.67 - ETA: 16:52 - loss: 0.9410 - mean_iou: 0.2270 - dice_coeff: 0.68 - ETA: 16:22 - loss: 0.9299 - mean_iou: 0.2368 - dice_coeff: 0.68 - ETA: 16:02 - loss: 0.9186 - mean_iou: 0.2453 - dice_coeff: 0.69 - ETA: 15:43 - loss: 0.9050 - mean_iou: 0.2524 - dice_coeff: 0.70 - ETA: 15:15 - loss: 0.8766 - mean_iou: 0.2589 - dice_coeff: 0.70 - ETA: 14:45 - loss: 0.8503 - mean_iou: 0.2658 - dice_coeff: 0.71 - ETA: 14:15 - loss: 0.8226 - mean_iou: 0.2725 - dice_coeff: 0.72 - ETA: 13:45 - lo

45/45 [==============================] - ETA: 21:31 - loss: 0.1454 - mean_iou: 0.7039 - dice_coeff: 0.95 - ETA: 21:21 - loss: 0.1512 - mean_iou: 0.7045 - dice_coeff: 0.95 - ETA: 20:53 - loss: 0.1393 - mean_iou: 0.7052 - dice_coeff: 0.95 - ETA: 20:10 - loss: 0.1383 - mean_iou: 0.7060 - dice_coeff: 0.95 - ETA: 19:30 - loss: 0.1384 - mean_iou: 0.7068 - dice_coeff: 0.95 - ETA: 18:59 - loss: 0.1471 - mean_iou: 0.7075 - dice_coeff: 0.95 - ETA: 18:22 - loss: 0.1413 - mean_iou: 0.7082 - dice_coeff: 0.95 - ETA: 17:47 - loss: 0.1446 - mean_iou: 0.7089 - dice_coeff: 0.95 - ETA: 17:14 - loss: 0.1431 - mean_iou: 0.7096 - dice_coeff: 0.95 - ETA: 16:43 - loss: 0.1471 - mean_iou: 0.7103 - dice_coeff: 0.95 - ETA: 16:13 - loss: 0.1462 - mean_iou: 0.7109 - dice_coeff: 0.95 - ETA: 15:47 - loss: 0.1459 - mean_iou: 0.7116 - dice_coeff: 0.95 - ETA: 15:19 - loss: 0.1441 - mean_iou: 0.7122 - dice_coeff: 0.95 - ETA: 14:50 - loss: 0.1437 - mean_iou: 0.7129 - dice_coeff: 0.95 - ETA: 14:21 - loss: 0.1434 - mean_io

45/45 [==============================] - ETA: 20:49 - loss: 0.1085 - mean_iou: 0.7876 - dice_coeff: 0.96 - ETA: 21:13 - loss: 0.0974 - mean_iou: 0.7879 - dice_coeff: 0.97 - ETA: 21:12 - loss: 0.0960 - mean_iou: 0.7882 - dice_coeff: 0.97 - ETA: 20:21 - loss: 0.0916 - mean_iou: 0.7885 - dice_coeff: 0.97 - ETA: 19:40 - loss: 0.0886 - mean_iou: 0.7888 - dice_coeff: 0.97 - ETA: 18:58 - loss: 0.0884 - mean_iou: 0.7891 - dice_coeff: 0.97 - ETA: 18:19 - loss: 0.0958 - mean_iou: 0.7894 - dice_coeff: 0.97 - ETA: 17:47 - loss: 0.0993 - mean_iou: 0.7897 - dice_coeff: 0.97 - ETA: 17:22 - loss: 0.1006 - mean_iou: 0.7900 - dice_coeff: 0.97 - ETA: 16:59 - loss: 0.1008 - mean_iou: 0.7902 - dice_coeff: 0.97 - ETA: 16:29 - loss: 0.0991 - mean_iou: 0.7905 - dice_coeff: 0.97 - ETA: 16:10 - loss: 0.0990 - mean_iou: 0.7907 - dice_coeff: 0.97 - ETA: 15:39 - loss: 0.0978 - mean_iou: 0.7910 - dice_coeff: 0.97 - ETA: 15:13 - loss: 0.0971 - mean_iou: 0.7913 - dice_coeff: 0.97 - ETA: 14:44 - loss: 0.0971 - mean_io

...

45/45 [==============================] - ETA: 21:18 - loss: 0.0283 - mean_iou: 0.9649 - dice_coeff: 0.99 - ETA: 22:07 - loss: 0.0321 - mean_iou: 0.9649 - dice_coeff: 0.99 - ETA: 22:07 - loss: 0.0303 - mean_iou: 0.9649 - dice_coeff: 0.99 - ETA: 21:41 - loss: 0.0299 - mean_iou: 0.9649 - dice_coeff: 0.99 - ETA: 21:12 - loss: 0.0293 - mean_iou: 0.9649 - dice_coeff: 0.99 - ETA: 20:45 - loss: 0.0313 - mean_iou: 0.9649 - dice_coeff: 0.99 - ETA: 20:10 - loss: 0.0312 - mean_iou: 0.9649 - dice_coeff: 0.99 - ETA: 19:39 - loss: 0.0313 - mean_iou: 0.9649 - dice_coeff: 0.99 - ETA: 19:13 - loss: 0.0315 - mean_iou: 0.9649 - dice_coeff: 0.99 - ETA: 18:36 - loss: 0.0310 - mean_iou: 0.9649 - dice_coeff: 0.99 - ETA: 18:01 - loss: 0.0308 - mean_iou: 0.9649 - dice_coeff: 0.99 - ETA: 17:27 - loss: 0.0304 - mean_iou: 0.9649 - dice_coeff: 0.99 - ETA: 16:54 - loss: 0.0301 - mean_iou: 0.9649 - dice_coeff: 0.99 - ETA: 16:19 - loss: 0.0300 - mean_iou: 0.9649 - dice_coeff: 0.99 - ETA: 15:47 - loss: 0.0303 - mean_io

45/45 [==============================] - ETA: 21:46 - loss: 0.0311 - mean_iou: 0.9650 - dice_coeff: 0.99 - ETA: 21:52 - loss: 0.0270 - mean_iou: 0.9650 - dice_coeff: 0.99 - ETA: 21:22 - loss: 0.0276 - mean_iou: 0.9650 - dice_coeff: 0.99 - ETA: 20:59 - loss: 0.0320 - mean_iou: 0.9650 - dice_coeff: 0.99 - ETA: 20:18 - loss: 0.0311 - mean_iou: 0.9650 - dice_coeff: 0.99 - ETA: 19:46 - loss: 0.0312 - mean_iou: 0.9650 - dice_coeff: 0.99 - ETA: 19:15 - loss: 0.0319 - mean_iou: 0.9650 - dice_coeff: 0.99 - ETA: 18:40 - loss: 0.0315 - mean_iou: 0.9650 - dice_coeff: 0.99 - ETA: 18:09 - loss: 0.0313 - mean_iou: 0.9650 - dice_coeff: 0.99 - ETA: 17:40 - loss: 0.0313 - mean_iou: 0.9650 - dice_coeff: 0.99 - ETA: 17:09 - loss: 0.0315 - mean_iou: 0.9650 - dice_coeff: 0.99 - ETA: 16:35 - loss: 0.0315 - mean_iou: 0.9650 - dice_coeff: 0.99 - ETA: 16:03 - loss: 0.0310 - mean_iou: 0.9650 - dice_coeff: 0.99 - ETA: 15:33 - loss: 0.0319 - mean_iou: 0.9650 - dice_coeff: 0.99 - ETA: 15:01 - loss: 0.0316 - mean_io

45/45 [==============================] - ETA: 20:59 - loss: 0.0300 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 20:20 - loss: 0.0317 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 19:55 - loss: 0.0306 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 19:32 - loss: 0.0344 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 19:02 - loss: 0.0340 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 18:35 - loss: 0.0355 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 18:07 - loss: 0.0341 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 17:41 - loss: 0.0326 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 17:11 - loss: 0.0332 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 16:43 - loss: 0.0324 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 16:13 - loss: 0.0320 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 15:44 - loss: 0.0324 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 15:15 - loss: 0.0324 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 14:47 - loss: 0.0322 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 14:19 - loss: 0.0320 - mean_io

45/45 [==============================] - ETA: 23:10 - loss: 0.0251 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 22:34 - loss: 0.0295 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 22:35 - loss: 0.0320 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 21:51 - loss: 0.0319 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 21:06 - loss: 0.0298 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 20:24 - loss: 0.0303 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 19:44 - loss: 0.0314 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 19:17 - loss: 0.0303 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 18:49 - loss: 0.0299 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 18:17 - loss: 0.0303 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 17:47 - loss: 0.0307 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 17:17 - loss: 0.0303 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 16:43 - loss: 0.0308 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 16:13 - loss: 0.0305 - mean_iou: 0.9651 - dice_coeff: 0.99 - ETA: 15:42 - loss: 0.0302 - mean_io

KeyboardInterrupt: 